# Local Search
## Solving N-queen problem with min-conflict algorithm

* First generate a complete assignment for all variables (this set of assignments may conflict)

* Repeat the following steps until there are no conflicts:

    * Randomly Select a variable that causes conflicts
    
    * Reassign the value of this variable to another value that with the least constraint onflicts with other variables

In [1]:
import numpy as np
import time
import random


def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def next_action(self, point):
        i, j = point
        if 0 <= i < self.n and 0 <= j < self.n and i * self.n + j < self.n ** 2 - 1:
            j += 1
            if j == self.n:
                j = 0
                i += 1
            return i, j
        else:
            return None

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

In [2]:
ran = random.SystemRandom()


class Queen():
    def __init__(self, num):
        self.num = num
        self.UConflict = []
        self.DConflict = []
        self.queen_array = []
        for i in range(2 * num + 1):
            self.DConflict.append(0)
            self.UConflict.append(0)
        for i in range(num + 1):
            self.queen_array.append(i)

    def partial_collision(self, col1, col2):
        row = self.queen_array[col2]
        tDown = self.num + col1 - row
        return self.UConflict[col1 + row] + self.DConflict[tDown]

    def partial_swap(self, col1, col2):
        row1 = self.queen_array[col1]
        row2 = self.queen_array[col2]

        self.queen_array[col1] = row2
        self.queen_array[col2] = row1

        # 计算冲突数
        tDown = self.num + col1 - row2
        self.DConflict[tDown] += 1
        self.UConflict[col1 + row2] += 1

    def swap(self, col1, col2):
        row1 = self.queen_array[col1]
        row2 = self.queen_array[col2]

        tDown1 = self.num + col1 - row1
        tDown2 = self.num + col2 - row2

        self.UConflict[row1 + col1] -= 1
        self.UConflict[row2 + col2] -= 1
        self.DConflict[tDown1] -= 1
        self.DConflict[tDown2] -= 1

        self.queen_array[col1] = row2
        self.queen_array[col2] = row1

        tDown1 = self.num + col1 - row2
        tDown2 = self.num + col2 - row1

        self.UConflict[row1 + col2] += 1
        self.UConflict[row2 + col1] += 1

        self.DConflict[tDown1] += 1
        self.DConflict[tDown2] += 1

    def initial_process(self):
        j = 1
        for i in range(int(3.08 * self.num)):
            m = ran.randint(j, self.num)
            if self.partial_collision(j, m) == 0:
                self.partial_swap(j, m)
                j += 1
                if j == self.num + 1:
                    return 0
        for i in range(j, self.num + 1):
            row = self.queen_array[i]
            tDown = self.num + i - row
            self.UConflict[row + i] += 1
            self.DConflict[tDown] += 1
        for i in range(j, self.num + 1):
            m = ran.randint(i, self.num)
            self.swap(i, m)
        return self.num - j + 1

    def total_collision(self, col):
        row = self.queen_array[col]
        tDown = self.num + col - row
        return self.UConflict[row + col] + self.DConflict[tDown] - 2

    def final_process(self, k):
        for i in range(self.num - k + 1, self.num + 1):
            if self.total_collision(i) > 0:
                isFound = False
                for j in range(1, self.num + 1):
                    self.swap(i, j)
                    if self.total_collision(i) > 0 or self.total_collision(j) > 0:
                        self.swap(i, j)
                    else:
                        isFound = True
                        break
                if not isFound:
                    self.__init__(self.num)
                    k = self.initial_process()
                    self.final_process(k)


def min_conflict(problem):
    action_stack = []

    queen = Queen(problem.n)
    left = queen.initial_process()
    queen.final_process(left)
    for i in range(1, len(queen.queen_array)):
        row, col = queen.queen_array[i], i
        action_stack.append((row - 1, col - 1))

        yield action_stack



In [4]:
# test block
n = 100
render = (n == 15)
p = Problem(n)
if render:
    import pygame

    w, h = 90 * n + 10, 90 * n + 10
    screen = pygame.display.set_mode((w, h))
    screen.fill('white')
    action_generator = min_conflict(p)
    clk = pygame.time.Clock()
    queen_img = pygame.image.load('./queen_s.png')
    queen_img = pygame.transform.scale(queen_img, (80, 80))
    while True:
        for event in pygame.event.get():
            if event == pygame.QUIT:
                exit()
        try:
            actions = next(action_generator)
            screen.fill('white')
            for i in range(n + 1):
                pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
            for action in actions:
                i, j = action
                screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
            pygame.display.flip()
        except StopIteration:
            pass
        clk.tick(5)
    pass
else:
    start_time = time.time()
    for actions in min_conflict(p):
        pass
    print(time.time() - start_time)
    p.print_state(actions)

0.017007112503051758
_________________________________________________________________________________________________________________________________________________________________________________________________________
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|